In [88]:
!pip install pyinterval

In [89]:
from interval import interval
import pandas as pd

In [90]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_middle_point(interv):
    return (interv[0].sup + interv[0].inf) / 2

def to_string(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

In [91]:
def calculate_volterra1(f, expected, k, y0, a, b, c, d, n):
    h = (b - a) / n
    x = []
    y = []
    exp = []
    x.append(a)
    y.append(y0)
    exp.append(expected(a))
    for i in range(1, n+1):
        x.append(a + i*h)
        sum = 0
        for j in range(i):
            sum += k(x[i], x[j], y[j])
        y.append(f(x[i]) + h*sum + (h*(x[i]-a)/2)*interval[0,h]*k(x[i], x[j], y[j]))
        exp.append(expected(x[i]))
    return x, y, exp

In [92]:
def calculate_volterra2(f, expected, k, kdf, y0, a, b, c, d, n):
    h = (b - a) / n
    x = []
    y = []
    exp = []
    x.append(a)
    y.append(y0)
    exp.append(expected(a))
    for i in range(1, n+1):
        x.append(a + i*h)
        sum = 0
        for j in range(i):
            sum += k(x[i], x[j], y[j])
        y.append(f(x[i]) + h*sum + (h*(x[i]-a)/2)*kdf(x[i], interval[a, x[i]], interval[c, d]))
        exp.append(expected(x[i]))
    return x, y, exp

In [93]:
a = 0
b = 1/4
c = 0
d = 2
n = 10
y0 = interval[1,1]
func = lambda x: 1
k = lambda x,s,y : y**2
kdf = lambda x,s,y : 2*y
expected = lambda x: 1/(1-x)

# Перша формула

In [97]:
tbl1 = pd.DataFrame(columns=['xi', 'yi', 'm(yi)', 'expected', 'e'])
x, y, exp = calculate_volterra1(func, expected, k, y0, a, b, c, d, n)
for i in range(n+1):
    mid_point=calculate_middle_point(y[i])
    tbl1.loc[len(tbl1)] = [f"{x[i]:9.4f}", to_string(y[i]), f"{mid_point:9.7f}", f"{exp[i]:9.7f}", abs(mid_point-exp[i])]
tbl1

,xi,yi,m(yi),expected,e
0,0.0000,"[1.0000000, 1.0000000]",1.0000000,1.0000000,0.000000
1,0.0250,"[1.0250000, 1.0250078]",1.0250039,1.0256410,0.000637
2,0.0500,"[1.0512656, 1.0512824]",1.0512740,1.0526316,0.001358
3,0.0750,"[1.0788946, 1.0789218]",1.0789082,1.0810811,0.002173
4,0.1000,"[1.1079949, 1.1080341]",1.1080145,1.1111111,0.003097
5,0.1250,"[1.1386863, 1.1387391]",1.1387127,1.1428571,0.004144
6,0.1500,"[1.1711014, 1.1711701]",1.1711358,1.1764706,0.005335
7,0.1750,"[1.2053884, 1.2054754]",1.2054319,1.2121212,0.006689
8,0.2000,"[1.2417124, 1.2418204]",1.2417664,1.2500000,0.008234
9,0.2250,"[1.2802587, 1.2803910]",1.2803248,1.2903226,0.009998


# Друга формула

In [100]:
tbl2 = pd.DataFrame(columns=['xi', 'yi', 'm(yi)', 'expected', 'e'])
x, y, exp = calculate_volterra2(func, expected, k, kdf, y0, a, b, c, d, n)
for i in range(n+1):
    mid_point=calculate_middle_point(y[i])
    tbl2.loc[len(tbl2)] = [f"{x[i]:9.4f}", to_string(y[i]), f"{mid_point:9.7f}", f"{exp[i]:9.7f}", abs(mid_point-exp[i])]
tbl2

,xi,yi,m(yi),expected,e
0,0.0000,"[1.0000000, 1.0000000]",1.0000000,1.0000000,0.000000
1,0.0250,"[1.0250000, 1.0262500]",1.0256250,1.0256410,0.000016
2,0.0500,"[1.0512656, 1.0538297]",1.0525477,1.0526316,0.000084
3,0.0750,"[1.0788946, 1.0828437]",1.0808691,1.0810811,0.000212
4,0.1000,"[1.1079949, 1.1134074]",1.1107012,1.1111111,0.000410
5,0.1250,"[1.1386863, 1.1456493]",1.1421678,1.1428571,0.000689
6,0.1500,"[1.1711014, 1.1797121]",1.1754068,1.1764706,0.001064
7,0.1750,"[1.2053884, 1.2157551]",1.2105718,1.2121212,0.001549
8,0.2000,"[1.2417124, 1.2539567]",1.2478345,1.2500000,0.002165
9,0.2250,"[1.2802587, 1.2945168]",1.2873878,1.2903226,0.002935
